# **Model Context Protocol**

_Het Model Context Protocol of MCP - uitgevonden door Anthropic - is een gestandaardiseerde manier om taalmodellen toegang te geven tot bepaalde tools om acties uit te voeren. Nadat OpenAI dit open-source framework ook is beginnen gebruiken, werd het universeel geadopteerd als het standaard communicatieprotocol voor agents en LLM's. Vaak wordt MCP beschreven als de "USB-C poort van taalmodellen"._

---

## **Voorbereiding**

In [47]:
%pip install fastmcp langchain_mcp_adapters langgraph

/usr/local/python/current/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=43798) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## **MCP server**

Voor de mensen die REST API's met HTTP endpoints kennen: MCP is hier een equivalent van maar dan met "tools". <br>

**Tools zijn acties** die iets **berekenen of uitvoeren** in opdracht van de LLM. <br>
We hebben dus een computer of **MCP server** nodig die luistert naar binnenkomende instructies en deze uitvoert. <br>
Welke tools er allemaal beschikbaar zijn en wat ze precies doen, wordt allemaal gedefinieerd op deze server. <br>

Er bestaan al veel **publieke MCP servers** die je kan gebruiken, bv. om het weerbericht op te vragen. <br>
Om de focus van deze workshop bij **privacy** te houden, gaan wij zelf één maken en hosten. <br>

Neem een kijkje naar de code van `mcp-server.py`. Probeer te beschrijven wat elke functie doet. 

**Start de MCP server** door het commando uit te voeren in de terminal:

```bash
python "3 - MCP/mcp-server.py"
```
<img src="../.github/fastmcp.png" alt="FastMCP" width="400"/>


---

## **MCP client**

Nu de server met onze tools draait, kunnen **MCP clients** ermee connecteren. <br>
Een client kan ook perfect met meerdere servers connecteren, dan heeft deze toegang tot de tools van alle servers.

### **1. Python script als client**
Een MCP client is typisch een taalmodel, maar we kunnen ook d.m.v. een Python script **manueel ermee verbinden** en tools aanspreken. <br>
Dit kan handig zijn om te **testen** of de server en zijn tools goed werken, alvorens we er een LLM op loslaten. <br>

In [1]:
from fastmcp import Client

config = {
    "mcpServers": {
        "private": {"url": "http://localhost:8000/sse"},    # Onze eigen lokale MCP server
    },
}

client = Client(config)

In [2]:
async with client:
    
    print("\nTools:")
    tools = await client.list_tools()
    for tool in tools:
        print(f"  - {tool.name}: {tool.description}")
        
    print("\nResources:")
    resources = await client.list_resources()
    for resource in resources:
        print(f"  - {resource.name}: {resource.description}")

    print("\nPrompts:")
    prompts = await client.list_prompts()
    for prompt in prompts:
        print(f"  - {prompt.name}: {prompt.description}")

    a = 5
    b = 3
    result = await client.call_tool("search_files", {"question": f"What can you tell me about blogpost.pdf?"})
    print(result.content[0].text)



Tools:
  - sum: Use this tool to calculate the sum of two numbers.
  - multiply: Use this tool to calculate the product of two numbers.
  - datetime: Use this tool to get the current date and time.
  - search_files: Use this tool to search for relevant information in the user's files and documents. Provide a search query or keywords to this tool.

Resources:

Prompts:
[./Blogpost.pdf - pagina 2]
Voorbeelden: op Ollama kan je van sommige taalmodellen - zoals de gemma3 
familie - ook QAT-varianten terugvinden die tot 3 keer sneller zijn 
(https://ollama.com/library/gemma34b-it-qat). Ook van computervisie 
modellen zoals YOLO bestaan PTQ-varianten zoals YOLOv8Detection-
Quantized (https://huggingface.co/qualcomm/YOLOv8Detection-Quantized).
Frameworks
Er bestaan verschillende Python-libraries die functies bevatten om jouw model 
te quantiseren:

[./Blogpost.pdf - pagina 4]
zoals DeepSeek-R1Distill-Qwen-7B (https://huggingface.co/deepseek-
ai/DeepSeek-R1Distill-Qwen-7B. Een ouder suc

### **2. Taalmodel als client**


Nu gaan we een taalmodel toegang geven tot de tools op onze MCP server. <br>

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

client = MultiServerMCPClient({"private": {"url": "http://localhost:8000/sse", "transport": "sse"}})
tools = await client.get_tools()

llm = ChatOllama(model="llama3.1")

agent = create_react_agent(llm, tools)

In [ ]:
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage

history = []
history.append({"role": "system", "content": "Your name is Milvus, you are a helpful assistant that answers questions. You have access to some tools to assist you. If you fetch context from the user's files, always include the source in your answer."})

while True:
    question = input("Type your next question (/bye to stop): ")
    if question.strip().lower() == "/bye":
        print("🤖 Milvus:\tGoodbye!")
        break
    if not question.strip():
        continue
    
    print(f"🙍‍♂️ User:\t{question}")
    history.append({"role": "user", "content": question})
    outputs = await agent.ainvoke({"messages": history})
    
    for output in outputs["messages"]:
        if isinstance(output, AIMessage):
            if output.tool_calls:
                print(f"🤖 Calling tool(s) {', '.join(tc['name'] for tc in output.tool_calls)}")
        #elif isinstance(output, ToolMessage):
            # print(f"🛠️ Tool:\t{output.content}")
            
    answer = outputs["messages"][-1].content
    print(f"🤖 Milvus:\t{answer}")
    history.append({"role": "assistant", "content": answer})

🙍‍♂️ User:	What is the sum of 5 and 3?
🤖 Calling tool(s) sum
🤖 Milvus:	The sum of 5 and 3 is 8.
